## Table of content
* [Library imports](#lib_imports)
* [Data import & Preprocessing](#Data_import_preprocessing)
* [Algorithm description & Helpers definition](#algorithm_description)

## Library imports <a class="anchor" id="lib_imports"></a>

In [1]:
import sys
sys.path.append("helpers.py")
from helpers import *

from collections import defaultdict
from pyspark.sql.types import IntegerType
import pandas as pd
from datetime import datetime
from datetime import timedelta
import pyspark.sql.functions as func
import numpy as np
from scipy import sparse
from numpy import array
import timeit
import itertools
import operator

%matplotlib inline
import matplotlib.pylab as plt

import pyspark
from pyspark.sql import SparkSession
import os
import getpass
from pyspark.streaming import StreamingContext
from  pyspark.streaming.kafka import KafkaUtils, OffsetRange

from branca.colormap import LinearColormap
import folium
import folium.plugins as plugins

In [2]:
MAX_TIME = pd.Timestamp(2150, 1, 1, 0, 0, 0)
MIN_TIME = pd.Timestamp(2000, 1, 1, 0, 0, 0)
MAX_WALK_DIST = 500
METERS_PER_MIN = 85
source_trip_id = 'origin_id'

## Imports and Pre-processing <a class="anchor" id="Data_import_preprocessing"></a>
In this section we will define generic functions for our imports and pre-process the data to remove unusable samples and format informations such as dates

In [3]:
# used not to perform main processing twice
# computes data structures from main data
reload_df = False;

In [4]:
if reload_df:
    username = getpass.getuser()

    # Use this when running on the cluster
    os.environ['PYSPARK_PYTHON'] = '/opt/anaconda3/bin/python'
    spark = (SparkSession
             .builder
             .appName('streaming-{0}'.format(username))
             .master('yarn')
             .config('spark.executor.memory', '1g')
             .config('spark.executor.instances', '2')
             .config('spark.executor.cores', '2')
             .config('spark.jars.packages', 'org.apache.spark:spark-streaming-kafka-0-8_2.11:2.3.2')
             .getOrCreate())

    sc = spark.sparkContext
    conf = sc.getConf()

    spark

We considered that delays are random variables modelling lateness per stop and trip id. 

In [5]:
if reload_df:
    delays = spark.read.parquet("hdfs://iccluster042.iccluster.epfl.ch:8020/user/hive/homes/dpetresc/"+\
                                "hive/sbb_delay_mean_avg_tripid_stopname/delta_0000001_0000001_0000")
    delays_df = delays.toPandas()
    delays_dict = defaultdict(lambda : defaultdict(lambda: {}))
    for iStop in delays_df.index.levels[0][:]:
        for jTrip in delays_df.loc[iStop].index[:]:
            delays_dict[iStop][jTrip] = delays_df.loc[(iStop, jTrip)].values
    delays = {k: dict(v) for k, v in delays_dict.items()}
    save_pkl(delays, 'delays_dict')
delays = load_pkl('delays_dict')

In [6]:
# renames columns in english names
# parses timestamps for trip_date and scheduled times
# removes samples for which the trip does not stop
# removes samples for which there is not at least one arrival and departure time (it be scheduled or observed)
def pre_process(raw):
    raw = raw.toDF(*new_columns)
    # parse timestamp
    raw = raw.withColumn('trip_date', func.to_timestamp(raw.trip_date, 'dd.MM.yyyy').alias('trip_date'))
    # remove trains that do not stop
    raw = raw.filter(func.col('stop_at_station')=='false').drop('stop_at_station')
    # Keep only trips that have at least one arrival and departure time
    raw = raw.filter((func.col('sch_arr_time').isNotNull() | func.col('real_arr_time').isNotNull()) & \
                     (func.col('sch_dep_time').isNotNull() | func.col('real_dep_time').isNotNull())
                    )
    # parse scheduled arrival and departure times
    raw = raw.withColumn('sch_arr_time', func.to_timestamp(raw.sch_arr_time, 'dd.MM.yyyyHH:mm').alias('sch_arr_time'))
    raw = raw.withColumn('sch_dep_time', func.to_timestamp(raw.sch_dep_time, 'dd.MM.yyyy HH:mm').alias('sch_dep_time'))
    
    # compute list of trips that appear more than once THIS IS SLOW, should try to do it on main data
    invalid_ids = raw.groupBy('trip_id').agg(func.count('*').alias('trip_id_counts')).filter(func.col('trip_id_counts')<=1)
    # put it as a list
    invalid_ids = invalid_ids.select('trip_id').distinct().rdd.map(lambda r: r[0]).collect()
    # remove trips for which there is only a single sample
    raw = raw.where(raw.trip_id.isin(invalid_ids) == False)
    raw = raw.withColumn('bpuic', raw['bpuic'].cast(IntegerType()))
    return raw

df = 0
if reload_df:
    df = pre_process(import_today_tomorrow(pd.Timestamp('2018-02-18'), spark))

Computes one-to-one mapping from stopnames to bpuics and bpuics to index between 0 and (N-1) where N is number of stops.

In [7]:
stop_bpuic_map = {}
bpuic_stop_map = {}
bpuic_index_map = {}
index_bpuic_map = {}

geo_data = pd.read_csv('geo_stops_data.csv', sep=';')

if reload_df:
    stop_bpuic = [t for t in df.select('bpuic', 'stop_name').distinct().collect() if t.bpuic in geo_data.bpuic.values]
    stop_bpuic_map, bpuic_stop_map, bpuic_index_map, index_bpuic_map = get_maps(stop_bpuic)
    save_pkl(stop_bpuic_map, 'sb_map')
    save_pkl(bpuic_stop_map, 'bs_map')
    save_pkl(bpuic_index_map, 'bi_map')
    save_pkl(index_bpuic_map, 'ib_map')
else: 
    stop_bpuic_map = load_pkl('sb_map')
    bpuic_stop_map = load_pkl('bs_map')
    bpuic_index_map = load_pkl('bi_map')
    index_bpuic_map = load_pkl('ib_map')

geo_data = geo_data[geo_data.bpuic.apply(lambda x: x in bpuic_index_map)]
geo_data['idx'] = geo_data.bpuic.apply(lambda x: bpuic_index_map[x])
geo_data.set_index('idx')
geo_data.head(2)

,bpuic,y_Coord_Est,x_Coord_Nord,idx
0,8508186,628463,220751,400
6,8590028,599934,200621,1882


Compute euclidian distance between two stops given by their index.

In [8]:
def dist(idx1, idx2):
    r1 = geo_data_indexed.loc[idx1]
    r2 = geo_data_indexed.loc[idx2]
    x1 = r1.x_Coord_Nord
    y1 = r1.y_Coord_Est
    x2 = r2.x_Coord_Nord
    y2 = r2.y_Coord_Est
    return euclidean_dist(x1, y1, x2, y2)

Compute dict of set of tuples (id, time) rpresenting for each stop the neighboors reachable by walk and the associated walking time.

In [9]:
%%time
num_bpuics = len(bpuic_index_map)
geo_data_indexed = geo_data.set_index('idx')

if reload_df:
    # find neighboors that have walking time less than 5 minutes
    bpuic_dists = defaultdict(lambda: set())
    for iB in range(num_bpuics)[:]:
        for jB in range(num_bpuics)[iB+1:]:
            d = dist(iB, jB)
            if d < MAX_WALK_DIST:
                bpuic_dists[iB].add((jB, np.ceil(d/METERS_PER_MIN)))
                bpuic_dists[jB].add((iB, np.ceil(d/METERS_PER_MIN)))
    save_pkl(dict(bpuic_dists), 'bpuic_dists')
else:
    bpuic_dists = load_pkl('bpuic_dists')

CPU times: user 11.2 ms, sys: 70 µs, total: 11.3 ms
Wall time: 10.3 ms


Load the connections either by increasing departure time or by decreasing arrival time.
A connection represent a direct link between two stops exiting in the timetable.

In [10]:
%%time
dep_filename = 'by_dep_connections'
arr_filename = 'by_arr_connections'
if reload_df:
    load_connections_to_pickle(dep_filename, arr_filename, df)   

dep_connections = load_pkl("./{}".format(dep_filename))
arr_connections = load_pkl("./{}".format(arr_filename))
dep_connections[:2]

CPU times: user 1.79 s, sys: 773 ms, total: 2.56 s
Wall time: 2.56 s


## Algorithm description & Helpers definition <a class="anchor" id="algorithm_description"></a>
In this section we will briefly discuss which function are needed in order to implement the journey planning algorithm as well as explain how we will implement it.

### Algorithm description
In order to create our own implementation of CSA (connection scan algorithm), we already computed all the connections from the timetable.
#### CSA 
CSA is a straightforward algorithm that computes the earliest arrival time at target t from source s given a list of connections in the form (stop_a, stop_b, departure_time, arrival_time) in linear time of the number of connections.
##### Reachable Stops
In CSA, we iterate over the connections in order of increasing departure time. We define a connection to be reachable if as we iterate over this connection, the earliest arrival time at stop_a is smaller than the departure_time of the connection or if we are already in the same trip earlier. The latter condition is a sub condition of the first in our implementation.
Moreover, we defined a trip as a succession of connections.
##### Arrival or Departure
Our implementation supports both the initial problem of finding earliest arrival time at target t and the reverse one of finding latest departure time from source s. We further explain only the earliest arrival time problem as the reverse is similar.
#### Intermediate results
Our first implement of the vanilla problem was able to compute earliest arrival time in half a second. We then extend our implementation to compute the corresponding journeys and then to consider also the probablity of never missing a connection during the journey.
#### Probability Tradeoff
Our fisrt idea was to consider optimal Pareto set of journeys with parameters probability of no miss, earliest arrival time, latest departure time, nb of legs. However, we quickly noticed that the task was a lot more complex than exepected (both implementation and mostly running time). We decided to choose the following tradeoff: at each stop we compute the earliest arrival time linked to the accumulated probability of no miss. We sliced what we considered reasonable probabilities (> 80%) as 2% slices.
While scanning a connection, we compute the probability of no miss for every slice for which the connection is reachable and then update the target earliest times according to the newly computed probabilities. 
We only update these probabilities if they have either better probability or smaller arrival time.
For example, we could have reached stop_a at 12:00 with probability 90% or at 12:10 w.p 95% or at 12:20 w.p 97% if we now scan a connection from stop_a to stop_b at 12:11 then the probability of no miss is p1 for slice 90%, p2 for slice 95% and 1 for slice 97%.
At stop_b, we now store at slice 90% * p1 with the connection departure_time and slice 95% * p2 with the same time. If the two results land on the same slice, we only keep the best probability of both.
Our implementation without further preprocessing ran in roughly 6 minutes.
#### Additional feature
Each time we update a stop earliest arrival times, we also update its neighbors defined as 5 minute close stops by walk. We compute the walk time between each neighbors and update their arrival time accounting for the walk delay

#### Filtering Preprocess
In order to speed our process up, we performed a few operations over the connection list.
1. We remove any connection that departs before our journey starts
2. We run the vanilla algorithm once and remove any connection that is not reachable before we run the heavier version
3. We add as a condition of reachability that a stop should be reachable in at most 5 legs
4. We limit our journey to 10h duration, this allows us to remove any connection departing 10 hours or more after the user-required departure time

#### Results
The last optimization make the heavy algorithm with probability tradeoff run under 2 minutes

In [11]:
%%time
# Retrieves the mean and standard deviation of the delay for a given trip at a given stop
def get_mean_std(trip, stop):
    vals = delays[stop][trip]
    mean, std = vals[0], vals[1]
    return mean, std
print(get_mean_std('80:06____:17400:000', 'Schaffhausen'))

(108.03488372093024, 131.40505880977503)
CPU times: user 119 µs, sys: 25 µs, total: 144 µs
Wall time: 147 µs


In [12]:
%%time
# computes the probability of missing a connection from trip_c1 to trip_c2 at stop_name. 
# offset_change accounts for an additional minute when changing trip at the same station
def p_miss(delta_t, trip_c1, trip_c2, stop_name, offset_change = 60, n_samples = 10000):
    #delta_t, trip_c1, trip_c2, stop_name = attr

    delta_t -= offset_change
    if stop_name not in delays:
        return 0.0
    if (trip_c1 not in delays[stop_name]) or (trip_c2 not in delays[stop_name]):
        return 0.0
    mean_d1, std_d1 = get_mean_std(trip_c1, stop_name)
    mean_d2, std_d2 = get_mean_std(trip_c2, stop_name)
    s = np.random.normal(mean_d1 - mean_d2, np.sqrt(std_d1*std_d1 + std_d2*std_d2), n_samples)
    return (s > delta_t).mean()
print(p_miss(120, '80:06____:17400:000', '80:06____:17402:000', 'Schaffhausen'))

0.3356
CPU times: user 953 µs, sys: 30 µs, total: 983 µs
Wall time: 859 µs


In [13]:
# Define probability slices, from min_p to 1.0 each bucket_size
min_p = 0.8
bucket_size = 0.02
proba_list_size = int((10 - 10*min_p)/(bucket_size*10)) + 1
proba_buckets = np.linspace(min_p, 1.0, proba_list_size)[:0:-1]
proba_buckets

array([1.  , 0.98, 0.96, 0.94, 0.92, 0.9 , 0.88, 0.86, 0.84, 0.82])

In [14]:
# helper that finds the index corresponding to the given probability in proba_buckets
def find_p_index(p):
    if p < min_p:
        return -1
    return proba_list_size - int(np.ceil((int(100*p)-int(100*min_p))/(100*bucket_size)))-1

print(find_p_index(0.98))
print(find_p_index(0.31))

1
-1


In [15]:
def update_neighbors(departure, op, id_, earl_times, tupl, bucket_idx, neighbors_dict):
    p_target, time_arr, trip_id, idx_source, footpath, iTrip_id, time_dep = tupl
    if id_ in neighbors_dict:
        neighbors_tuples = neighbors_dict[id_]
        for (n_id, n_t) in neighbors_tuples:
            new_arr_time = time_arr + timedelta(minutes = n_t) if \
                departure else time_arr - timedelta(minutes = n_t)
            neighbor_tupl = earl_times[n_id][bucket_idx]

            if op(new_arr_time, neighbor_tupl[1]):
                    if (bucket_idx != 0 and op(new_arr_time, earl_times[n_id][bucket_idx-1][1]))\
                                or bucket_idx==0:
                            earl_times[n_id][bucket_idx] = (p_target, new_arr_time, trip_id, idx_source,\
                                                            True, iTrip_id, time_dep)

In [16]:
# Computes a boolean flag for each connection for the preprocessing steps
def get_flags(departure, connections, start, source, neighbors_dict):
    
    limit_temp = MAX_TIME if departure else MIN_TIME
    connections_min_time = min(connections, key = lambda t: t[2])
    time = start.replace(year = connections_min_time[2].year)\
        .replace(month = connections_min_time[2].month)\
        .replace(day = connections_min_time[2].day)
    MAX_MIN_STAMP = time + pd.Timedelta(hours=10) if departure else time - pd.Timedelta(hours=10)
    earl_times = [limit_temp]*len(bpuic_index_map)
    valid_connections = [True]*len(connections)
    earl_times[source] = time
    list_trip = [(0, 0)]*len(bpuic_index_map)
    list_trip[source] = (0, source_trip_id)
    
    op = operator.le if departure else operator.ge
    
    for idx_c, iConnection in enumerate(connections):
        
        if int(iConnection[0]) in bpuic_index_map and int(iConnection[1]) in bpuic_index_map:
            
            idx_source = bpuic_index_map[int(iConnection[0])]
            idx_target = bpuic_index_map[int(iConnection[1])]
            idx_tmp = idx_source
            idx_source = idx_source if departure else idx_target
            idx_target = idx_target if departure else idx_tmp
            legs, arr_trip = list_trip[idx_source]

            time_dep = iConnection[2]
            time_arr = iConnection[3]
            time_tmp = time_dep
            time_dep = time_dep if departure else time_arr
            time_arr = time_arr if departure else time_tmp
            #print(time, time_dep, op(time, time_dep))
            trip_id = iConnection[4]
            if op(time, time_dep) and op(earl_times[idx_source], time_dep) \
                and (op(time_dep, MAX_MIN_STAMP)) and (legs <=5):

                if op(time_arr, earl_times[idx_target]):
                    earl_times[idx_target] = time_arr

                    if arr_trip != trip_id:
                        list_trip[idx_target] = (legs+1, trip_id)
                    else:
                        list_trip[idx_target] = (legs, trip_id)

                    if idx_target in neighbors_dict:
                        neighbors_tuples = neighbors_dict[idx_target]
                        for (n_id, n_t) in neighbors_tuples:
                            new_arr_time = time_arr + timedelta(minutes = n_t) if departure \
                                else time_arr - timedelta(minutes = n_t)
                            neighbor_time = earl_times[n_id]

                            if op(new_arr_time, neighbor_time):
                                earl_times[n_id] = new_arr_time
                                list_trip[n_id] = (legs, trip_id)
            else:
                valid_connections[idx_c] = False
        else:
            valid_connections[idx_c] = False
    return valid_connections

In [17]:
# Per connection update loop
def update_time(time, earliest_arrival, iConnection, departure=True):
    if int(iConnection[0]) in bpuic_index_map and int(iConnection[1]) in bpuic_index_map:
        
        idx_source = bpuic_index_map[int(iConnection[0])]
        idx_target = bpuic_index_map[int(iConnection[1])]
        idx_tmp = idx_source
        idx_source = idx_source if departure else idx_target
        idx_target = idx_target if departure else idx_tmp

        op = operator.le if departure else operator.ge
        time_dep = iConnection[2]
        time_arr = iConnection[3]
        time_tmp = time_dep
        time_dep = time_dep if departure else time_arr
        time_arr = time_arr if departure else time_tmp
        trip_id = iConnection[4]
        #print(time, time_dep)
        if op(time, time_dep):
            l = [tupl for tupl in earliest_arrival[idx_source] if op(tupl[1], time_dep)]
            for iTuple in l:
                iTrip_id = iTuple[2]
                source_time = iTuple[1]
                if iTrip_id != trip_id and iTrip_id != source_trip_id:
                    #print(2)
                    # compute miss proba
                    delta_t = time_dep - source_time if departure else source_time - time_dep
                    delta_t = delta_t.seconds
                    p = 1-p_miss(delta_t, trip_id, iTrip_id, bpuic_stop_map[index_bpuic_map[idx_source]])
                    p_target = p*iTuple[0]
                else:
                    #print(3)
                    p_target = iTuple[0]
                p_bucket_idx = find_p_index(p_target)
                if p_bucket_idx == -1:
                    continue;
                target_tuple = earliest_arrival[idx_target][p_bucket_idx]
                if op(time_arr, target_tuple[1]):
                    if (p_bucket_idx != 0 and op(time_arr, earliest_arrival[idx_target][p_bucket_idx-1][1]))\
                        or p_bucket_idx==0:
                        res_tupl = (p_target, time_arr, trip_id, idx_source, False, iTrip_id, time_dep)
                        earliest_arrival[idx_target][p_bucket_idx] = res_tupl
                        update_neighbors(departure, op, idx_target, earliest_arrival, res_tupl, p_bucket_idx, bpuic_dists)

In [18]:
# initialise structures then loops over connections

def find_earliest_time(departure, connections, limit_temp, \
                         start, dest, start_stamp):
    origin = start if departure else dest
    flags = get_flags(departure, connections[:], start_stamp, origin, bpuic_dists)
    filtered_connections = list(map(lambda x: x[0], filter(lambda x: x[1], zip(connections, flags))))
    
    connections_min_time = min(filtered_connections, key = lambda t: t[2]) #if departure \
        #else max(filtered_connections, key = lambda t: t[2])
    time = start_stamp.replace(year = connections_min_time[2].year)\
        .replace(month = connections_min_time[2].month)\
        .replace(day = connections_min_time[2].day)
    
    earliest_arrival = [[(1.0, limit_temp, '-', 0, False, '-', limit_temp) for j in range(proba_list_size)] \
                            for i in range(len(bpuic_index_map))]
      
    
    earliest_arrival[origin][0] = (1.0, time, source_trip_id, origin, False, source_trip_id, time)
    #list_trip[origin] = (-1,-1,-1)
    for iConnection in filtered_connections:
        update_time(time, earliest_arrival, iConnection, departure)
    return earliest_arrival

We then process the journeys found.

In [19]:
def get_infos_path(elem):
    return elem[0], elem[1], elem[2], elem[3], elem[4], elem[5], elem[6]

In [20]:
def find_trip_idx(list_path, trip):
    for idx, elem in enumerate(list_path):
        proba, arr, trip_id, idx_source, by_foot, trip_id_origin, dep = get_infos_path(elem)
        if trip_id == trip:
            return idx

In [21]:
def delet_loop_in_path(path):    
    final_path = []
    for i,p in enumerate(path):
        seen_stops = {}
        temp_path = []
        for idx, connection in enumerate(p):
            if connection[0] not in seen_stops:
                seen_stops[connection[0]] = idx
                temp_path.append(connection)
            else: 
                temp_path = temp_path[:seen_stops[connection[0]]]
                temp_path.append(connection)
        final_path.append(temp_path)
    return final_path

In [22]:
def find_path(departure, earliest_arrival):
    path = [[] for i in range(10)]
    
    if departure : 
        #We start from the target and go back to find the source (the correct path for all the proba)
        for idx, elems in enumerate(earliest_arrival[target][:6]):
            connections_seen = []
            temp_localisation = target
            temp_trip_idx = -1
            proba_t, arr_t, trip_id_t, idx_source_t, by_foot_t, trip_id_origin_t, dep_t = get_infos_path(elems)
            if trip_id_t != '-': 
                while temp_localisation != source:
                    if temp_trip_idx == -1:
                        arr = arr_t
                        proba = proba_t
                        trip_id = trip_id_t
                        trip_id_origin = trip_id_origin_t
                        idx_source = idx_source_t
                        dep = dep_t
                        by_foot = by_foot_t
                    else:
                        idx_new_target = find_trip_idx(earliest_arrival[temp_localisation], temp_trip_idx)
                        proba, arr, trip_id, idx_source, by_foot, trip_id_origin, dep = get_infos_path(earliest_arrival[temp_localisation][idx_new_target])

                    s = bpuic_stop_map[index_bpuic_map[idx_source]]
                    t = bpuic_stop_map[index_bpuic_map[temp_localisation]]
                    path[idx].append((s, t, dep, arr, trip_id, proba))  
                    temp_localisation = idx_source
                    temp_trip_idx = trip_id_origin

        path = [i[::-1] for i in path[:]]
        
    else:
        # Same but we need to inverse the source and the arrival point (add the correct new trip_id and time) 
        for idx, elems in enumerate(earliest_arrival[source][:6]):
            connections_seen = []
            temp_localisation = source
            temp_trip_idx = -1
            proba_t, arr_t, trip_id_t, idx_target_t, by_foot_t, trip_id_origin_t, dep_t = get_infos_path(elems)
            if trip_id_t != '-': 
                while temp_localisation != target:
                    if temp_trip_idx == -1:
                        arr = arr_t
                        proba = proba_t
                        trip_id = trip_id_t
                        trip_id_origin = trip_id_origin_t
                        idx_target = idx_target_t
                        dep = dep_t
                        by_foot = by_foot_t
                    else:
                        idx_new_target = find_trip_idx(earliest_arrival[temp_localisation], temp_trip_idx)
                        proba, arr, trip_id, idx_target, by_foot, trip_id_origin, dep = get_infos_path(earliest_arrival[temp_localisation][idx_new_target])

                    s = bpuic_stop_map[index_bpuic_map[temp_localisation]]
                    t = bpuic_stop_map[index_bpuic_map[idx_target]]
                    path[idx].append((s, t, dep, arr, trip_id, proba)) 
                    temp_localisation = idx_target
                    temp_trip_idx = trip_id_origin
                    
    return delet_loop_in_path(path)

Here we can query with the wanted parameters.

In [23]:
%%time

# User parameters for the problem are
# Source, Target, departure/arrival time, departure(true/false)

dep_stop_name = 'Genève'
arr_stop_name = 'Lausanne'

# whether to perform earliest arrival time or latest departure time problem
departure = True

user_departure_time = pd.Timestamp(2019, 3, 15, 15, 10)
user_arrival_time = pd.Timestamp(2019, 3, 15, 15, 43)


source = bpuic_index_map[stop_bpuic_map[dep_stop_name]]
target = bpuic_index_map[stop_bpuic_map[arr_stop_name]]

if departure:
    earliest_arrival = find_earliest_time(departure, dep_connections[:], MAX_TIME, \
                                                        source,  target, user_departure_time)
else:
    earliest_arrival = find_earliest_time(departure, arr_connections, MIN_TIME, \
                                                        source,  target, user_arrival_time)
    
final_path = find_path(departure, earliest_arrival)

CPU times: user 1min 45s, sys: 854 ms, total: 1min 46s
Wall time: 1min 46s


In [24]:
journeys = list(filter(lambda x : x != [], final_path))

We display the different journeys using folium. A round represents a leg and the color represents the probability of the journey.

In [ ]:
#Source : https://github.com/ValentinMinder/Swisstopo-WGS84-LV03/blob/master/scripts/py/wgs84_ch1903.py

# Convert CH y/x to WGS lat
def CHtoWGSlat(y, x):
    # Axiliary values (% Bern)
    y_aux = (y - 600000) / 1000000
    x_aux = (x - 200000) / 1000000
    lat = (16.9023892 + (3.238272 * x_aux)) + \
            - (0.270978 * pow(y_aux, 2)) + \
            - (0.002528 * pow(x_aux, 2)) + \
            - (0.0447 * pow(y_aux, 2) * x_aux) + \
            - (0.0140 * pow(x_aux, 3))
    # Unit 10000" to 1" and convert seconds to degrees (dec)
    lat = (lat * 100) / 36
    return lat

# Convert CH y/x to WGS long
def CHtoWGSlng(y, x):
    # Axiliary values (% Bern)
    y_aux = (y - 600000) / 1000000
    x_aux = (x - 200000) / 1000000
    lng = (2.6779094 + (4.728982 * y_aux) + \
            + (0.791484 * y_aux * x_aux) + \
            + (0.1306 * y_aux * pow(x_aux, 2))) + \
            - (0.0436 * pow(y_aux, 3))
    # Unit 10000" to 1" and convert seconds to degrees (dec)
    lng = (lng * 100) / 36
    return lng

def swissTopo_to_WGS84(bpuic, y_east, x_north, idx) :
    lat = CHtoWGSlat(y_east, x_north)
    lng = CHtoWGSlng(y_east, x_north)
    return  pd.Series([bpuic, lat, lng, int(idx)])
    
#bpuic, #latitude, #longitude, #idx
wgs84_geo_data = geo_data.apply(lambda row : swissTopo_to_WGS84(row[0], row[1], row[2], row[3]), axis=1)
wgs84_geo_data.columns = ['bpuic', 'latitude', 'longitude', 'idx']
wgs84_geo_data.bpuic = wgs84_geo_data.bpuic.astype('int64')
wgs84_geo_data.idx = wgs84_geo_data.idx.astype('int64')
wgs84_geo_data['stop_name'] = wgs84_geo_data.bpuic.map(lambda x : bpuic_stop_map[x])

In [ ]:
def print_journey_step(journey, n):
    #Display journey informations
    s1 = "Journey {} : {} ---> {}. \n".format(n, journey[0][0], journey[-1][0])
    s2 = "Departure time : {} - Arrival time : {} \n". format(journey[0][1] , journey[-1][1])
    print(s1 + "\t" + s2)
    if len(journey)>2:
        print("Journey details :")
        for i in range(0, len(journey), 2):
            A = journey[i]
            B = journey[i+1]
            print('\t'+'{} : {} --> {} : {} with trip_id = {}'.format(A[0], A[1].time(), B[1].time(), B[0], A[2]))
    print('--------------------------------------------------------------')

In [ ]:
def draw_path(stations, path, m, color_scale, i) :
    all_stops = list(map(lambda x : x[0], path))
    all_stops.append(path[-1][1])
    
    proba = path[-1][5] #proba of success for this trip
    
    n = 0 #Counter of stops we iterate over
    prev = all_stops[0] #store previous stop
    points = [] #list of points to store to draw polyline
    
    trips_ids = list(map(lambda x : x[4], path))
    trips_taken = [] #store the different trips to detect change of connection
    
    journey_step = [] #changes to be displayed
    
    color_stop = color_scale(proba)
    
    
    #Iterate over each stop and draw a new stop at each change of connection
    for p in all_stops :
        #Additionnal information to be displayed over the points
        features_current = stations[stations['stop_name'] == p]
        pop = "{}, success_rate : {}%"\
            .format(features_current.iloc[0,4], proba*100)
        right = [features_current.iloc[0,1], features_current.iloc[0,2]]
        
        
        if(n == 0) : #first stop
            trips_taken.append(trips_ids[n])
            folium.CircleMarker(location=right, color = color_stop, \
                                popup=pop, fill=True, radius=10).add_to(m)
            journey_step.append([p, path[n][2], trips_ids[n]])
        else :
            if(n == (len(all_stops) - 1)) : #last stop
                folium.CircleMarker(location=right, color = color_stop, \
                                    popup=pop, fill=True, radius=10).add_to(m)
                journey_step.append([p, path[n-1][3], trips_ids[n-1]])
            else :
                if(n < (len(all_stops) -2) and trips_ids[n] not in trips_taken) :
                    #Change of connection
                    trips_taken.append(trips_ids[n])
                    times = ", arrival : {}, departure : {}".format(path[n-1][3].time(), path[n][2].time())
                    folium.CircleMarker(location=right, color= color_stop, \
                                        popup=pop + times, fill=True, radius=5).add_to(m)
                    
                    #update changes to display later
                    journey_step.append([p, path[n-1][3], trips_ids[n-1]])
                    journey_step.append([p, path[n][2], trips_ids[n]])
                prev = p
            
        features_prev = stations[stations['stop_name'] == prev]
        left = [features_prev.iloc[0,1], features_prev.iloc[0,2]]
        points.append(left)
        points.append(right)
        n += 1
    folium.PolyLine(locations=points, color=color_scale(proba), weight=5).add_to(m)
        
    print_journey_step(journey_step, i)

In [ ]:
def draw_map(stations, path_list):    
    #set the colorscale matching the probabilities of a successful journey
    
    color_scale = LinearColormap(['r', 'y', 'g'], vmin = 0.82, vmax = 1)
    color_scale.caption = 'Colormap of journey success rate'
    
    #create the folium map that will contain all journeys
    depart = stations[stations['stop_name'] == path_list[0][0][0]]
    depart_lat = depart['latitude'].iloc[0]
    depart_lon = depart['longitude'].iloc[0]
    arr = stations[stations['stop_name'] == path_list[0][-1][1]]
    arr_lat = arr['latitude'].iloc[0]
    arr_lon = arr['longitude'].iloc[0]
    m = folium.Map(location=[np.mean([depart_lat, arr_lat]), np.mean([depart_lon, arr_lon])], \
                       zoom_start=10,tiles='CARTODBPOSITRON')
    
    #some folium parameters (minimap, layercontrol, full screen, etc)
   
    mcg = plugins.MarkerCluster(control=False)
    m.add_child(mcg)
    m.add_child(plugins.MiniMap())
    
    m.add_child(color_scale)
    plugins.Fullscreen(
        position='topright',
        title='Expand me',
        title_cancel='Exit me',
        force_separate_button=True
    ).add_to(m)
    
    #iterate over the journey list, display each journey on the folium map
    i = 1
    for p in path_list :
        g = plugins.FeatureGroupSubGroup(mcg, 'journey {}'.format(i))
        m.add_child(g)
        draw_path(stations, p, g, color_scale, i)
        i += 1
        
    folium.LayerControl(Collapsed = False).add_to(m)
    return m

In [ ]:
m = draw_map(wgs84_geo_data, journeys)
m

In [ ]:
m.save('journeys_Lausanne_Geneve.html')